# 课时106 tensorflow自定义层

In [1]:
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.3.1


## 1. 自定义全连接层(dense)

In [7]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        # 首先继承父类tf.keras.layers.Layer的属性
        super(Linear, self).__init__()
        # 初始化权重
        # add_weights方法继承自父类的方法
        w_init = tf.random_normal_initializer()
        self.weight = tf.Variable(initial_value=w_init(shape=(input_dim, units),                                                              dtype='float32'),
                                  trainable=True)
        b_init = tf.zeros_initializer()
        self.bias = tf.Variable(initial_value=b_init(shape=(units,),dtype='float32'), 
                                trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias

In [8]:
x = tf.ones(shape=(2,2))
hidden_layer = Linear(units=4, input_dim=2)
y = hidden_layer(inputs=x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.00104036,  0.00183899, -0.00479256, -0.07462753],
       [ 0.00104036,  0.00183899, -0.00479256, -0.07462753]],
      dtype=float32)>

In [12]:
hidden_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.06415289, -0.10509486, -0.05924907,  0.08822808],
        [-0.07400787,  0.00171867, -0.04386234, -0.12518638]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

## 2. 为层添加权重
>还可以使用add_weights的方法为层更加快捷的添加权重

In [5]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        # 首先继承父类tf.keras.layers.Layer的属性
        super(Linear, self).__init__()
        # 初始化权重
        # add_weight方法继承自父类的方法
        self.weight = self.add_weight(shape=(input_dim, units),
                                      initializer='random_normal',
                                      trainable=True)
        self.bias = self.add_weight(shape=(units,),
                                      initializer='zeros',
                                      trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias

In [6]:
x = tf.ones(shape=(2,2))
hidden_layer = Linear(units=4, input_dim=2)
y = hidden_layer(inputs=x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.08450989,  0.06303281,  0.02029446, -0.02093529],
       [ 0.08450989,  0.06303281,  0.02029446, -0.02093529]],
      dtype=float32)>

## 3. 推迟权重创建到输入shape知晓以后

In [3]:
x = tf.keras.layers.Dense(units=64)
x.weights

[]

因此我们可以在我们的自定义层中加入build(self, input_shape)方法中创建层的权重

In [9]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        # 首先继承父类tf.keras.layers.Layer的属性
        super(Linear, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        # 初始化权重
        # add_weight方法继承自父类的方法
        # input_shape = [batch_num, data_dim], input_shape[-1] = data_dim
        self.weight = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='random_normal',
                                      trainable=True)
        self.bias = self.add_weight(shape=(self.units,),
                                      initializer='zeros',
                                      trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias

In [6]:
hidden_layer = Linear(units=4)
hidden_layer.weights

[]

In [7]:
x = tf.ones(shape=(2,2))
y = hidden_layer(inputs=x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.07163899, -0.03818507, -0.04971079,  0.06779702],
       [ 0.07163899, -0.03818507, -0.04971079,  0.06779702]],
      dtype=float32)>

In [8]:
hidden_layer.weights

[<tf.Variable 'linear/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.07849186, -0.00700583, -0.06317079,  0.07516818],
        [-0.00685288, -0.03117924,  0.01346   , -0.00737116]],
       dtype=float32)>,
 <tf.Variable 'linear/Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

## 4. 层可以递归组合
>如果将一个层实例分配为另一个层的特性，则外部层将开始追踪内部层的权重。我们建议在init()方法中创建此类子层(由于子层通常具有构建方法，它们将与外部层同时构建)

In [10]:
class MLP_Block(tf.keras.layers.Layer):
    def __init__(self):
        super(MLP_Block, self).__init__()
        # Linear就是子层
        self.linear_1 = Linear(units=32)
        self.linear_2 = Linear(units=64)
        self.linear_3 = Linear(units=1)
    
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        x = self.linear_3(x)
        return x

In [11]:
mlp = MLP_Block()

In [12]:
class MLP_Block2(tf.keras.layers.Layer):
    def __init__(self):
        super(MLP_Block, self).__init__()
        # Linear就是子层
        self.linear_1 = tf.keras.layers.Dense(units=32)
        self.linear_2 = tf.keras.layers.Dense(units=64)
        self.linear_3 = tf.keras.layers.Dense(units=32)
    
    def call(self, inputs):
        x1 = self.linear_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.linear_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.linear_3(x2)
        return tf.concat([x1, x3])

In [ ]:
mlp = MLP_Block2()

## 5. 自定义模型

In [13]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        # Linear就是子层
        self.linear_1 = tf.keras.layers.Dense(units=32)
        self.linear_2 = tf.keras.layers.Dense(units=64)
        self.linear_3 = tf.keras.layers.Dense(units=32)
    
    def call(self, inputs):
        x1 = self.linear_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.linear_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.linear_3(x2)
        return tf.concat([x1, x3])

In [14]:
model = MLP_Model()

继承自Model相比继承自Layers多出的一些方法：

In [16]:
model.fit

<bound method Model.fit of <__main__.MLP_Model object at 0x00000133543A47C8>>

In [17]:
model.save

<bound method Model.save of <__main__.MLP_Model object at 0x00000133543A47C8>>